# Check monotonicity of results
This notebook is not an EDA notebook but rather a tool to check if the results in the vehicle_data table are monotonically decreasing.  

## Setup

### Import

In [ ]:
import plotly.express as px

from core.sql_utils import *
from core.pandas_utils import *
from transform.results.main import *

### Data extraction

In [ ]:
results = get_all_processed_results()

### Check

In [ ]:
def soh_is_monotonically_decreasing(df):
    return df.set_index("date")["soh"].is_monotonic_decreasing

montonic_mask_per_vin = (
    results
    .groupby("vin")
    .apply(soh_is_monotonically_decreasing, include_groups=False)
    .to_frame("monotonically_decreasing")
    .reset_index(drop=False)
)
montonic_mask_per_vin#.query("~monotonically_decreasing")

In [ ]:
value_counts_per_vin = {
    "normalized": montonic_mask_per_vin["monotonically_decreasing"].value_counts(dropna=False, normalize=True),
    "absolute": montonic_mask_per_vin["monotonically_decreasing"].value_counts(dropna=False, normalize=False)
}
pd.concat(value_counts_per_vin, axis=1, keys=value_counts_per_vin.keys(), names=["value_counts_type"])